In [28]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
import warnings
import numpy as np

In [6]:
df = pd.read_csv(r"C:\Users\KAVISH\Downloads\FidelFolio_Final_DF.csv")

In [7]:
for col in df.columns:
    if col not in ['Company', 'Year']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [8]:
df1 = df[df['Year']< 2024 ].drop([' Target 2 ', ' Target 3 '], axis=1)

In [12]:
exog_cols = [col for col in df1.columns if col not in ['Company', 'Year', ' Target 1 ']]

In [9]:
df1.sort_values(['Company', 'Year'], inplace=True)


In [10]:
df_sarimax= df1

In [20]:
from sklearn.model_selection import train_test_split

In [33]:
df = df_sarimax.copy()
df = df[['Year', ' Target 1 ', 'Company']].dropna()

# Sort by time and optionally company
df = df.sort_values(['Year', 'Company'])

# Create a datetime index (or numeric index)
df['time_index'] = pd.factorize(df['Year'])[0]

# Group by year to get a pooled total/average (or you can keep full granularity)
# Here we just use the average Target 1 per year across companies
grouped_df = df.groupby('time_index').agg({' Target 1 ': 'mean'}).reset_index()

# Split into train/val (hold out last year)
train = grouped_df[grouped_df['time_index'] < grouped_df['time_index'].max()]
val = grouped_df[grouped_df['time_index'] == grouped_df['time_index'].max()]

# Define endog
endog = train[' Target 1 '].values

# --- 2. Fit SARIMAX ---
model = SARIMAX(endog, order=(1, 1, 1), seasonal_order=(1, 1, 0, 4),
                enforce_stationarity=False, enforce_invertibility=False)
results = model.fit(disp=False)

# --- 3. Predict for validation period ---
forecast = results.predict(start=len(endog), end=len(endog))

# --- 4. Evaluate ---
actual = val[' Target 1 '].values
rmse = np.sqrt(mean_squared_error(actual, forecast))
print(f"Validation RMSE: {rmse:.2f}")

# --- 5. Plot with Plotly ---
fig = go.Figure()

# Plot training actuals
fig.add_trace(go.Scatter(
    x=train['time_index'],
    y=train[' Target 1 '],
    mode='lines+markers',
    name='Train Actual'
))

# Plot validation actuals
fig.add_trace(go.Scatter(
    x=val['time_index'],
    y=actual,
    mode='markers+lines',
    name='Validation Actual',
    line=dict(dash='dot', color='blue')
))

# Plot validation predictions
fig.add_trace(go.Scatter(
    x=val['time_index'],
    y=forecast,
    mode='markers+lines',
    name='Validation Forecast (SARIMAX)',
    line=dict(dash='dash', color='red')
))

fig.update_layout(
    title="Pooled SARIMAX Forecast vs Actual",
    xaxis_title="Time Index (Year Encoded)",
    yaxis_title="Target 1",
    height=500
)

fig.show()
    

Validation RMSE: 50.84


In [35]:
from sklearn.metrics import mean_squared_error

In [36]:
rmse = np.sqrt(mean_squared_error(val_df[' Target 1 '], val_df['SARIMAX_Val_Pred']))
print(f"Validation RMSE: {rmse:.2f}")

Validation RMSE: 1505218.15


In [ ]:
val_df.head(60)

,Company,Year,Target 1,SARIMAX_Val_Pred
0,3M India,2019,4.91,72.131057
1,3M India,2020,-18.12,-15.281069
2,3M India,2021,-52.44,31.208096
3,3M India,2022,18.77,60.103117
4,3M India,2023,7.21,68.148893
5,3i Infotech,2019,-39.30,-65.873161
6,3i Infotech,2020,385.64,-19.087125
7,3i Infotech,2021,-45.77,15.826902
8,3i Infotech,2022,-44.60,-78.328532
9,63 Moons Tech.,2019,-15.74,46.951553


: 